In [1]:
import numpy as np
import random
import math
# import matplotlib.pyplot as plt

In [21]:
H = np.array([[1, 1, 0, 1, 0, 0], 
              [0, 1, 1, 0, 1, 0], 
              [1, 0, 0, 0, 1, 1], 
              [0, 0, 1, 1, 0, 1]])


In [48]:
## Signal Parameters

c = np.array([0, 0, 1, 0, 1, 1])
y = np.array([1, 0, 1, 0, 1, 1])
p = 0.2
L0 = (1-p)/p  #y=0
L1 = p/(1-p)  #y=1
lnL0 = math.log(L0)
lnL1 = math.log(L1)
r = (2*y-1)*lnL1 

print("Switching Probability:", p)
print("Transmitted Signal:", c)
print("Received Signal:", y)
print("Log-likelihood:", lnL0)
print("a priori log likelihood ratios:", r)

Switching Probability: 0.2
Transmitted Signal: [0 0 1 0 1 1]
Received Signal: [1 0 1 0 1 1]
Log-likelihood: 1.3862943611198906
a priori log likelihood ratios: [-1.38629436  1.38629436 -1.38629436  1.38629436 -1.38629436 -1.38629436]


In [49]:
msg_node = 6 # bit_node
chk_node = 4 # parity node

## Iteration parameters
max_iter = 3
curr_iter = 0

In [50]:
# message from bit node to chk node
M = np.zeros([4,6])

# message from chk node to bit node
E = np.zeros([4,6])

L = np.zeros(r.shape)
z = np.zeros(r.shape)

# for j in range(chk_node):
for i in range(msg_node):
    M[:,i] = H[:,i] * r[i]
       
# for j in range(chk_node):
#     for i in range(msg_node):
#         M[j,i] = H[j,i] * r[i]
#         # vectorize
print(M)

# Is vectorization computationally expensive for sparse matrices?

[[-1.38629436  1.38629436 -0.          1.38629436 -0.         -0.        ]
 [-0.          1.38629436 -1.38629436  0.         -1.38629436 -0.        ]
 [-1.38629436  0.         -0.          0.         -1.38629436 -1.38629436]
 [-0.          0.         -1.38629436  1.38629436 -0.         -1.38629436]]


In [54]:
while True:
tanhM = np.tanh(M/2)
    for j in range(chk_node):
        t = 1
        for i in range(msg_node):
            if H[j,i]:
                t *= tanhM[j,i]
        
        # modify for sparse matrices 
        # store indices in hashmap for quick access
        for i in range(msg_node):
            if H[j,i]:
                prod_tanhM = t/tanhM[j,i]
                E[j,i] = np.log((1 + prod_tanhM)/(1 - prod_tanhM))
            else:
                E[j,i] = 0
        
    for i in range(msg_node):
        L[i] = r[i] + np.sum(E[:,i])
        z[i] = 0 if L[i]>0 else 1
#     z = [0 if L[i]>0 else 1 for i in range(msg_node)]
    
    if curr_iter < max_iter or np.all(np.dot(H,z)%2==0):
        break
    else:
        for i in range(msg_node):
            t = np.sum(E[:,i])
            for j in range(chk_node):
                if H[j,i]==0:
                    M[j,i] = (t - E[j,i]) + r[i]
    
    curr_iter += 1

print("No of iterations run :", curr_iter)
print("tanhM values:", tanhM)
print("log-likelihood values :", L)
print("Decoded Signal :", z)
print(E)


[[-0.6  0.6 -0.   0.6 -0.  -0. ]
 [-0.   0.6 -0.6  0.  -0.6 -0. ]
 [-0.6  0.  -0.   0.  -0.6 -0.6]
 [-0.   0.  -0.6  0.6 -0.  -0.6]]
[ 4.64388006  1.38629436 -7.41646878  1.38629436 -1.38629436 -1.38629436]
[0 0 1 0 1 1]
0 [0 0 0 0]
break
[[ 0.7537718 -0.7537718  0.        -0.7537718  0.         0.       ]
 [ 0.         0.7537718 -0.7537718  0.        -0.7537718  0.       ]
 [ 0.7537718  0.         0.         0.         0.7537718  0.7537718]
 [ 0.         0.        -0.7537718  0.7537718  0.        -0.7537718]]
[0.7537718 0.        0.7537718 0.       ]
